In [2]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("full_pcd_30000_samples.npz")
with open("label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
num_classes

5

In [3]:
train_dataset[0][1][0]

tensor(1)

In [4]:
from src.model import PoseWithClassModel

model = PoseWithClassModel(num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [5]:
from src.train_utils import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model, train_loader, val_loader, optimizer, device, epochs=15)

Epoch 1/15 - Total Loss: 1092.3025 - Cls: 596.8071 - Pose: 495.4954
Validation - Loss: 4.5852 | Cls Loss: 747.9843 | Pose Loss: 114.0251 | Accuracy: 24.12%
Epoch 2/15 - Total Loss: 696.6442 - Cls: 227.6527 - Pose: 468.9915
Validation - Loss: 1.2804 | Cls Loss: 125.8268 | Pose Loss: 114.8932 | Accuracy: 76.13%
Epoch 3/15 - Total Loss: 631.6485 - Cls: 172.3029 - Pose: 459.3456
Validation - Loss: 0.9320 | Cls Loss: 65.6754 | Pose Loss: 109.5449 | Accuracy: 85.22%
Epoch 4/15 - Total Loss: 608.2617 - Cls: 152.1085 - Pose: 456.1532
Validation - Loss: 0.9708 | Cls Loss: 68.0930 | Pose Loss: 114.4264 | Accuracy: 84.55%
Epoch 5/15 - Total Loss: 578.9013 - Cls: 124.9336 - Pose: 453.9677
Validation - Loss: 1.3359 | Cls Loss: 138.8635 | Pose Loss: 112.2833 | Accuracy: 72.93%


KeyboardInterrupt: 